### The object of this project is to make a comparison of the "classical" knn method and a new method proposed by [Dend et al., 2015] (Deng, Z., Zhu, X., Cheng, D., Zong, M., and Zhang, S. (2016). Efficient knn classification algorithm for big data. Neurocomputing, 195:143–148. Learning for Medical Imaging.)

### To make the comparison, we will use two datasets: GISETTE and OPTDIGITS.
+ GISETTE is is a handwritten digit recognition problem. The problem is to
separate the highly confusible digits ’4’ and ’9’. This dataset is one of five datasets of
the NIPS 2003 feature selection challenge
+ OPTDIGITS is a handwritten digit recognition problem. 

### We will use the "classical" KNN method from the library scikit-learn and we will develop the new method using the library pyspark.

In [ ]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pyspark import SparkContext
sc=SparkContext(master='local[2]')

In [ ]:
 #lanch UI
sc

<SparkContext master=local[2] appName=pyspark-shell>

In [ ]:
#Create a spark session
from pyspark.sql import SparkSession

In [ ]:
spark=SparkSession.builder.appName("AdvancedTopicML").getOrCreate()

In [1]:
import numpy as np
import pandas as pd

# Data retrieving and preprocessing

## Gisette dataset

In [2]:
# read the file gisette_db/gisette-train.data
# and convert it to a pandas dataframe
gisette_data = pd.read_csv('gisette-db/gisette_train.data', sep=' ', header=None)
gisette_data

,0,1,2,3,4,5,6,7,8,9,...,4991,4992,4993,4994,4995,4996,4997,4998,4999,5000
0,550,0,495,0,0,0,0,976,0,0,...,0,0,991,991,0,0,0,0,983,NaN
1,0,0,0,0,0,0,0,976,0,0,...,475,0,991,0,0,991,0,0,0,NaN
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
3,0,0,742,0,0,0,0,684,0,956,...,0,0,0,0,0,674,0,0,838,NaN
4,0,0,0,0,0,0,0,608,0,979,...,0,0,828,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,783,0,0,0,NaN
5996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,921,0,886,0,NaN
5997,0,0,0,0,0,758,0,0,0,522,...,901,0,0,0,0,980,0,0,0,NaN
5998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,690,0,0,0,0,0,NaN


In [3]:
gisette_label = pd.read_csv('gisette-db/gisette_train.labels', sep=' ', header=None)

In [4]:
gisette_label

,0
0,1
1,-1
2,1
3,1
4,1
...,...
5995,-1
5996,1
5997,-1
5998,-1


In [5]:
# replace Nan with 0
gisette_data = gisette_data.fillna(0)

## Optdigit dataset

In [6]:
# import optdigits-db/optdigits.tra
# and convert it to a pandas dataframe
optdigits_data = pd.read_csv('optdigits-db/optdigits.tra', sep=',', header=None)
optdigits_data

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,0,1,6,15,12,1,0,0,0,7,...,0,0,0,6,14,7,1,0,0,0
1,0,0,10,16,6,0,0,0,0,7,...,0,0,0,10,16,15,3,0,0,0
2,0,0,8,15,16,13,0,0,0,1,...,0,0,0,9,14,0,0,0,0,7
3,0,0,0,3,11,16,0,0,0,0,...,0,0,0,0,1,15,2,0,0,4
4,0,0,5,14,4,0,0,0,0,0,...,0,0,0,4,12,14,7,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3818,0,0,5,13,11,2,0,0,0,2,...,0,0,0,8,13,15,10,1,0,9
3819,0,0,0,1,12,1,0,0,0,0,...,0,0,0,0,4,9,0,0,0,4
3820,0,0,3,15,0,0,0,0,0,0,...,0,0,0,4,14,16,9,0,0,6
3821,0,0,6,16,2,0,0,0,0,0,...,0,0,0,5,16,16,16,5,0,6


In [7]:
optdigits_label = optdigits_data[64]
optdigits_data = optdigits_data.drop(64, axis=1)
optdigits_label

0       0
1       0
2       7
3       4
4       6
       ..
3818    9
3819    4
3820    6
3821    6
3822    7
Name: 64, Length: 3823, dtype: int64

# Making a "classical" KNN model for comparison

### We will use 4 metrics to compare the two methods: accuracy, precision, recall and F1-score. We will also measure the time

+ optdigits dataset

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [9]:
optdigit_train_data, optdigit_test_data, optdigit_train_label, optdigit_test_label = train_test_split(optdigits_data, optdigits_label, test_size=0.2, random_state=42)

In [10]:
knn = KNeighborsClassifier(n_neighbors=3)

In [11]:
knn.fit(optdigit_train_data, optdigit_train_label)

KNeighborsClassifier(n_neighbors=3)

In [12]:
# prediction
# record the time
import time
start_opt = time.time()
predict = knn.predict(optdigit_test_data)
end_opt = time.time()


### Accuracy, precision, recall and F1-score

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [14]:
acc_opt = accuracy_score(optdigit_test_label, predict)
prec_opt = precision_score(optdigit_test_label, predict, average='macro')
rec_opt = recall_score(optdigit_test_label, predict, average='macro')
f1_opt = f1_score(optdigit_test_label, predict, average='macro')


In [15]:
print('Accuracy: ', acc_opt, '\nPrecision', prec_opt,
      '\nRecall', rec_opt, '\nF1', f1_opt, '\nTime', end_opt-start_opt, 's')


Accuracy:  0.984313725490196 
Precision 0.9847688494865668 
Recall 0.9848506588806535 
F1 0.984685110583763 
Time 0.07819581031799316 s


+ GISETTE dataset

In [16]:
gisette_train_data, gisette_test_data, gisette_train_label, gisette_test_label = train_test_split(gisette_data, gisette_label, test_size=0.2, random_state=42)

In [17]:
knn = KNeighborsClassifier(n_neighbors=3)

In [18]:
knn.fit(gisette_train_data, gisette_train_label)

c:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=3)

In [19]:
start_gisette = time.time()
predict = knn.predict(gisette_test_data)
end_gisette = time.time()


In [20]:
len(predict)


1200

In [21]:
acc_gisette = accuracy_score(gisette_test_label, predict)
prec_gisette = precision_score(gisette_test_label, predict, average='macro')
rec_gisette = recall_score(gisette_test_label, predict, average='macro')
f1_gisette = f1_score(gisette_test_label, predict, average='macro')


In [22]:
print('Accuracy: ', acc_gisette, '\nPrecision', prec_gisette,
      '\nRecall', rec_gisette, '\nF1', f1_gisette, '\nTime', end_gisette-start_gisette, 's')


Accuracy:  0.9691666666666666 
Precision 0.9700124611978327 
Recall 0.9694444521615442 
F1 0.9691618482054487 
Time 0.7807705402374268 s


# Visualizing the results of the classical KNN models

In [36]:
# make a dataframe to store the results of gisette
results = pd.DataFrame(columns=['Accuracy', 'Precision', 'Recall', 'F1', 'Time'])
results.loc['Gisette classical KNN'] = [acc_gisette, prec_gisette, rec_gisette, f1_gisette, end_gisette-start_gisette]
results.loc['Optdigits classical KNN'] = [acc_opt, prec_opt, rec_opt, f1_opt, end_opt-start_opt]

In [37]:
results

,Accuracy,Precision,Recall,F1,Time
Gisette classical KNN,0.969167,0.970012,0.969444,0.969162,0.780771
Optdigits classical KNN,0.984314,0.984769,0.984851,0.984685,0.078196


In [40]:
results.to_latex('results-classical-KNN.tex')

C:\Users\HP\AppData\Local\Temp\ipykernel_16112\305113872.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  results.to_latex('results-classical-KNN.tex')


# KNN classification for Big Data with Spark

### The [Deng et al., 2015] algortihm is the following:

### - Produce m cluster centers using LSC algorithm, denoted by C = {c1, c2, ..., cm}

In [45]:
from sklearn.cluster import SpectralClustering

In [54]:
sp = SpectralClustering(n_clusters=20, random_state=42)

+ For Gisette dataset

In [55]:
sp.fit(gisette_train_data)

c:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\manifold\_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


SpectralClustering(n_clusters=20, random_state=42)

In [56]:
# group the indices with similar labels in the same cluster
cluster = {}
for i in range(len(sp.labels_)):
    if sp.labels_[i] not in cluster:
        cluster[sp.labels_[i]] = [i]
    else:
        cluster[sp.labels_[i]].append(i)


In [59]:
cluster[3]

[0,
 1,
 2,
 3,
 4,
 6,
 8,
 9,
 10,
 11,
 13,
 16,
 17,
 18,
 20,
 22,
 23,
 24,
 25,
 27,
 28,
 30,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 59,
 60,
 61,
 63,
 64,
 65,
 66,
 67,
 69,
 70,
 73,
 74,
 75,
 76,
 78,
 79,
 81,
 83,
 84,
 85,
 86,
 87,
 89,
 92,
 93,
 95,
 96,
 97,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 108,
 110,
 111,
 117,
 119,
 120,
 122,
 123,
 124,
 125,
 129,
 130,
 131,
 133,
 135,
 137,
 138,
 139,
 140,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 150,
 151,
 153,
 155,
 156,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 166,
 167,
 169,
 170,
 172,
 173,
 174,
 176,
 177,
 178,
 181,
 182,
 184,
 185,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 199,
 200,
 201,
 204,
 205,
 206,
 207,
 208,
 210,
 211,
 213,
 214,
 216,
 217,
 218,
 221,
 222,
 223,
 224,
 225,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 235,
 236,
 238,
 240,
 241,
 242,
 243,
 244,
 245,
 2

In [60]:
for i in range(len(cluster)):
    print('Cluster ', i, 'has ', len(cluster[i]), 'elements')

Cluster  0 has  22 elements
Cluster  1 has  229 elements
Cluster  2 has  1 elements
Cluster  3 has  3608 elements
Cluster  4 has  1 elements
Cluster  5 has  97 elements
Cluster  6 has  47 elements
Cluster  7 has  15 elements
Cluster  8 has  174 elements
Cluster  9 has  46 elements
Cluster  10 has  68 elements
Cluster  11 has  9 elements
Cluster  12 has  1 elements
Cluster  13 has  7 elements
Cluster  14 has  135 elements
Cluster  15 has  36 elements
Cluster  16 has  1 elements
Cluster  17 has  233 elements
Cluster  18 has  64 elements
Cluster  19 has  6 elements


In [61]:
centers = []
for i in range(len(cluster)):
    centers.append(np.mean(gisette_train_data.iloc[cluster[i]], axis=0))

centers

[0        39.227273
 1         0.000000
 2       225.818182
 3         0.000000
 4         0.000000
            ...    
 4996    344.136364
 4997     54.590909
 4998    177.818182
 4999    199.090909
 5000      0.000000
 Length: 5001, dtype: float64,
 0       165.301310
 1         0.000000
 2       168.266376
 3         0.000000
 4        14.056769
            ...    
 4996    212.825328
 4997      8.633188
 4998    122.602620
 4999    160.104803
 5000      0.000000
 Length: 5001, dtype: float64,
 0         0.0
 1         0.0
 2       979.0
 3         0.0
 4         0.0
         ...  
 4996      0.0
 4997      0.0
 4998    511.0
 4999    960.0
 5000      0.0
 Length: 5001, dtype: float64,
 0       105.002494
 1        11.806541
 2       173.562084
 3         1.339800
 4        14.213415
            ...    
 4996    266.480876
 4997     16.502772
 4998    127.734756
 4999    157.918514
 5000      0.000000
 Length: 5001, dtype: float64,
 0         0.0
 1         0.0
 2         0.0
 3    

In [65]:
len(centers)

20

+ For Optdigits dataset

In [66]:
sp = SpectralClustering(n_clusters=20, random_state=42)

In [67]:
sp.fit(optdigit_train_data)

c:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\manifold\_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


KeyboardInterrupt: 

In [ ]:
cluster_opt = {}
for i in range(len(sp.labels_)):
    if sp.labels_[i] not in cluster_opt:
        cluster_opt[sp.labels_[i]] = [i]
    else:
        cluster_opt[sp.labels_[i]].append(i)

In [ ]:
centers_opt = []
for i in range(len(cluster_opt)):
    centers_opt.append(np.mean(optdigit_train_data.iloc[cluster_opt[i]], axis=0))

In [ ]:
centers_opt

### - Compute the distance D(y,Ci) between each test sample y and each cluster center,denoted by D(y,Ci), i = 1,2,…,m;

In [68]:
from math import dist

+ For Gisette dataset

In [69]:
distances = {}
for i in range(gisette_test_data.shape[0]):
    distances[i] = []
    for j in range(len(centers)):
        distances[i].append((j, dist(gisette_test_data.iloc[i], centers[j])))

In [70]:
distances

{0: [(0, 16473.60812158006),
  (1, 15702.879015853418),
  (2, 24504.369447100653),
  (3, 15798.649268785564),
  (4, 23128.624948318913),
  (5, 15822.114883643753),
  (6, 16106.001578946374),
  (7, 16558.396791691855),
  (8, 15694.276191839006),
  (9, 15936.392664937617),
  (10, 15859.083786688332),
  (11, 16952.252414529637),
  (12, 28096.951756373855),
  (13, 17748.85838352336),
  (14, 15694.59775780727),
  (15, 15929.531059302722),
  (16, 20620.854298500824),
  (17, 15792.867677084929),
  (18, 16204.11157903814),
  (19, 17531.02558877958)],
 1: [(0, 17980.711662974434),
  (1, 17574.44832263718),
  (2, 25596.10616871246),
  (3, 17492.90206751291),
  (4, 25233.398641483076),
  (5, 17440.09325318764),
  (6, 17605.002869390162),
  (7, 17605.255385190463),
  (8, 17592.598270249004),
  (9, 17560.912034271605),
  (10, 17529.139934615112),
  (11, 17824.38930334044),
  (12, 28416.97874510941),
  (13, 19351.047858038117),
  (14, 17434.35898495858),
  (15, 17577.590758701866),
  (16, 24522.2226

+ For Optdigits dataset

In [ ]:
distances_opt = {}
for i in range(optdigit_test_data.shape[0]):
    distances_opt[i] = []
    for j in range(len(centers_opt)):
        distances_opt[i].append((j, dist(optdigit_test_data.iloc[i], centers_opt[j])))

### Compute the nearest cluster center Ci to y, Ci = min{D(y,Ci)}, i = 1,2,…,m;

+ For Gisette dataset

In [71]:
for i in range(len(distances)):
    distances[i] = sorted(distances[i], key=lambda x: x[1])
    

In [73]:
prediction_gisette = []
temps = []
for i in range(len(distances)):
    knn = KNeighborsClassifier(n_neighbors=3)
    deb = time.time()
    knn.fit(gisette_train_data.iloc[cluster[distances[i][0][0]]], gisette_train_label.iloc[cluster[distances[i][0][0]]])
    prediction_gisette.append(knn.predict(gisette_test_data.iloc[i].values.reshape(1, -1))[0])
    end = time.time()
    temps.append(end-deb)


c:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWa

In [74]:
acc_gis = accuracy_score(gisette_test_label, prediction_gisette)
prec_gis = precision_score(gisette_test_label, prediction_gisette, average='macro')
rec_gis = recall_score(gisette_test_label, prediction_gisette, average='macro')
f1_gis = f1_score(gisette_test_label, prediction_gisette, average='macro')

In [75]:
res_gis_spectral = [acc_gis, prec_gis, rec_gis, f1_gis, np.mean(temps)]

In [78]:
print('Accuracy: ', acc_gis, '\nPrecision', prec_gis,
        '\nRecall', rec_gis, '\nF1', f1_gis, '\nTime', np.mean(temps), 's')

Accuracy:  0.8825 
Precision 0.8842332396627861 
Recall 0.8829021172326234 
F1 0.8824313366347984 
Time 0.10052430073420207 s


+ For Optdigits dataset

In [79]:
for i in range(len(distances_opt)):
    distances_opt[i] = sorted(distances_opt[i], key=lambda x: x[1])

NameError: name 'distances_opt' is not defined

In [80]:
prediction_opt = []
temps_opt = []
for i in range(len(distances_opt)):
    knn = KNeighborsClassifier(n_neighbors=3)
    deb = time.time()
    knn.fit(optdigit_train_data.iloc[cluster_opt[distances_opt[i][0][0]]], optdigit_train_label.iloc[cluster_opt[distances_opt[i][0][0]]])
    prediction_opt.append(knn.predict(optdigit_test_data.iloc[i].values.reshape(1, -1))[0])
    end = time.time()
    temps_opt.append(end-deb)

NameError: name 'distances_opt' is not defined

In [ ]:
res_opt_spectral = [acc_opt, prec_opt, rec_opt, f1_opt, np.mean(temps_opt)]

In [ ]:
print('Accuracy: ', acc_opt, '\nPrecision', prec_opt,
        '\nRecall', rec_opt, '\nF1', f1_opt, '\nTime', np.mean(temps_opt), 's') 

In [81]:
results.loc['Gisette spectral KNN'] = res_gis_spectral
results

,Accuracy,Precision,Recall,F1,Time
Gisette classical KNN,0.969167,0.970012,0.969444,0.969162,0.780771
Optdigits classical KNN,0.984314,0.984769,0.984851,0.984685,0.078196
Gisette spectral KNN,0.882500,0.884233,0.882902,0.882431,0.100524


In [82]:
# Select the first and the third rows of the dataframe and export to latex
results.iloc[[0, 2]].to_latex('results-spectral-KNN.tex')


C:\Users\HP\AppData\Local\Temp\ipykernel_16112\1455597016.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  results.iloc[[0, 2]].to_latex('results-spectral-KNN.tex')
